In [1]:
import os
os.environ['HTTP_PROXY'] = "http://127.0.0.1:10809"
os.environ['HTTPS_PROXY'] = "http://127.0.0.1:10809"

In [2]:
import ee
import geemap

Map = geemap.Map()
Map

In [4]:
point = ee.Geometry.Point([-122.4439, 37.7538])

image = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
    .filterBounds(point) \
    .filterDate('2016-01-01', '2016-12-31') \
    .sort('CLOUD_COVER') \
    .first() \
    .select('B[1-7]')

vis_params = {
    'min': 0,
    'max': 3000,
    'bands': ['B5', 'B4', 'B3']
}

Map.centerObject(point, 8)
Map.addLayer(image, vis_params, "Landsat-8")

Check image properties

In [5]:
# 影像的属性信息
print('Date:',ee.Date(image.get('system:time_start')).format('YYYY-MM-dd').getInfo())
print('cloud_cover:',image.get('CLOUD_COVER').getInfo())

Date: 2016-11-18
cloud_cover: 0.08


In [6]:
nlcd = ee.Image('USGS/NLCD/NLCD2016').select('landcover').clip(image.geometry())
Map.addLayer(nlcd, {}, 'NLCD')

# Make the training dataset.
points = nlcd.sample(**{
    'region': image.geometry(),
    'scale': 30,
    'numPixels': 5000,
    'seed': 0,
    'geometries': True  # Set this to False to ignore geometries
})

Map.addLayer(points, {}, 'training', False)

print(points.size().getInfo())
print(points.first().getInfo())

In [8]:
bands = ['B1','B2', 'B3', 'B4', 'B5', 'B6', 'B7']
label = 'landcover'

# Overlay the points on the imagery to get training.
training = image.select(bands).sampleRegions(**{
  'collection': points,
  'properties': [label],
  'scale': 30
})

In [ ]:
# 做精度评价
# 在training要素集中增加一个random属性，值为0到1的随机数
withRandom = training.randomColumn('random')

#把生成随机数大于0.7的分成测试数据，否则为训练数据。这样可以把数据分成两份
split = 0.7
trainingPartition = withRandom.filter(ee.Filter.lt('random', split))
testingPartition = withRandom.filter(ee.Filter.gte('random', split))

# 利用70%数据随机森林进行训练
trainedClassifier = ee.Classifier.smileRandomForest(5).train(trainingPartition,label,bands)
#对Landsat-8进行分类
result=image.select(bands).classify(trainedClassifier)

Map.addLayer(result.randomVisualizer(), {}, 'classfied')

In [10]:
class_values = nlcd.get('landcover_class_values').getInfo()
class_palette = nlcd.get('landcover_class_palette').getInfo()
landcover = result.set('classification_class_values', class_values)
landcover = landcover.set('classification_class_palette', class_palette)

Map.addLayer(landcover, {}, 'Land cover')

print('Change layer opacity:')
cluster_layer = Map.layers[-1]
cluster_layer.interact(opacity=(0, 1, 0.1))

Map.add_legend(builtin_legend='NLCD')

Map(bottom=25611.0, center=[37.75379999999999, -122.44390000000001], controls=(WidgetControl(options=['positio…

In [14]:
train_accuracy = trainedClassifier.confusionMatrix()

train_accuracy.getInfo()

print('accuracy:',train_accuracy.accuracy().getInfo())
print('kappa:',train_accuracy.kappa().getInfo())
print('producersAccuracy:',train_accuracy.producersAccuracy().getInfo())
print('consumersAccuracy:',train_accuracy.consumersAccuracy().getInfo())

In [16]:
validated = testingPartition.classify(trainedClassifier)

test_accuracy = validated.errorMatrix('landcover', 'classification')

print('accuracy:',test_accuracy.accuracy().getInfo())
print('kappa:',test_accuracy.kappa().getInfo())
print('producersAccuracy:',test_accuracy.producersAccuracy().getInfo())
print('consumersAccuracy:',test_accuracy.consumersAccuracy().getInfo())

In [ ]:
import csv
import os

out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
training_csv = os.path.join(out_dir, 'train_accuracy.csv')
testing_csv = os.path.join(out_dir, 'test_accuracy.csv')

with open(training_csv, "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerows(train_accuracy.getInfo())
    
with open(testing_csv, "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerows(test_accuracy.getInfo())